In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

# mostly a duplicate of the circle_routing...

_n_paths_         = 50
_n_circles_       = 100
_radius_min_      = 20
_radius_max_      = 30
_circle_geoms_    = []
_min_circle_sep_  = 40
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 20                     # less than the _min_circle_sep_

def circleOverlaps(cx, cy, r):
    for _geom_ in _circle_geoms_:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    _max_attempts_ = 100
    attempts  = 0
    cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_),     random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
    while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
        cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_), random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
        attempts += 1
    if attempts == _max_attempts_:
        return None
    return cx, cy, r

for i in range(_n_circles_):
    to_unpack = findOpening()
    if to_unpack is not None:
        cx, cy, r = to_unpack
        _circle_geoms_.append((cx,cy,r))

# points to connect
_pts_ = []
c0    = random.randint(0, len(_circle_geoms_)-1)
c1    =  random.randint(0, len(_circle_geoms_)-1)
while c1 == c0:
    c1 =  random.randint(0, len(_circle_geoms_)-1)
a0, a1 = random.random() * 2 * pi, random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c0]
_pts_.append((cx+(r+_radius_start_+1)*cos(a0), cy+(r+_radius_start_+1)*sin(a0)))
_pts_.append((cx+(r+_escape_px_)*cos(a0), cy+(r+_escape_px_)*sin(a0)))
cx, cy, r = _circle_geoms_[c1]
_pts_.append((cx+(r+_escape_px_)*cos(a1), cy+(r+_escape_px_)*sin(a1)))
_pts_.append((cx+(r+_radius_start_+1)*cos(a1), cy+(r+_radius_start_+1)*sin(a1)))

entries         = [_pts_]
_exit_segments_ = []
_exit_segments_.append(_pts_[2:])
for i in range(_n_paths_):
    _more_pts_ = [_pts_[0], _pts_[1]]
    c          = random.randint(0,len(_circle_geoms_)-1)
    while c == c0:
        c  = random.randint(0,len(_circle_geoms_)-1)
    a = random.random() * 2 * pi
    cx, cy, r = _circle_geoms_[c]
    _more_pts_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
    _more_pts_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))
    _exit_segments_.append(_more_pts_[2:])
    entries.append(_more_pts_)

In [ ]:
def calculatePathAroundCircles(pts, circle_geoms, radius_inc_test, half_sep):
    def breakSegment(_segment_):
        if rt.segmentLength(_segment_) < 2.0:
            return _segment_
        for _geom_ in circle_geoms:
            _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
            _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
            if _dist_ <= _circle_plus_[2]:
                if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                    dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                    l        = sqrt(dx*dx+dy*dy)
                    dx,  dy  = dx/l, dy/l
                    pdx, pdy = -dy, dx 
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+half_sep), _geom_[1] + pdy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
                else:
                    dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                    l      = sqrt(dx*dx+dy*dy)
                    dx, dy = dx/l, dy/l
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0]  + dx*(_geom_[2]+half_sep), _geom_[1]  + dy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
        return _segment_

    last_length = 0
    _segments_  = []
    for _pt_ in pts:
        _segments_.append(_pt_)
    while last_length != len(_segments_):
        last_length    = len(_segments_)
        _new_segments_ = []
        for i in range(len(_segments_)-1):
            _new_ = breakSegment([_segments_[i], _segments_[i+1]])
            if len(_new_) == 3:
                _new_segments_.append(_new_[0])
                _new_segments_.append(_new_[1])
            else:
                _new_segments_.append(_new_[0])
        _new_segments_.append(_new_[-1])
        _segments_ = _new_segments_
        
    return _segments_

all_paths = []
for x in entries:
    _segments_ = calculatePathAroundCircles(x, _circle_geoms_, _radius_inc_test_, _half_sep_)
    all_paths.append(_segments_)

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    for i in range(len(_path_)-1):
        svg += f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />'
# Render Entry Points
svg += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    svg += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg += '</svg>'

svg2 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg2 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    _path_ = rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, ampends=10.0, amp=10.0)
    for i in range(len(_path_)-1):
        svg2 += f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />'
# Render Entry Points
svg2 += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg2 += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    # svg2 += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg2 += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg2 += '</svg>'

rt.tile([svg,svg2])

In [ ]:
class XYQuadTree(object):
    #
    # __init__()
    # - bounds = (xmin,ymin,xmax,ymax)
    #
    def __init__(self, bounds, max_pts_per_node=50):
        self.bounds           = bounds
        self.max_pts_per_node = max_pts_per_node
        self.node_pts         = {}
        self.node_pts['']     = set()
        self.node_bounds      = {}
        self.node_bounds['']  = bounds

    #
    # __split__() - split a quad... maybe...
    #
    def __split__(self, q):
        b = self.node_bounds[q]
        xs, ys = [],[]
        for pt in self.node_pts[q]:
            xs.append(pt[0]), ys.append(pt[1])
        xs, ys = sorted(xs), sorted(ys)
        mid = int(len(xs)/2)
        if xs[0] == xs[-1]:
            xsplit = (b[0] + b[2])/2.0
        else:
            xsplit = xs[mid]
        if ys[0] == ys[-1]:
            ysplit = (b[1] + b[3])/2.0
        else:
            ysplit = ys[mid]

        self.node_pts   [q+'0'] = set()
        self.node_bounds[q+'0'] = (b[0],    b[1],     xsplit,  ysplit)
        self.node_pts   [q+'1'] = set()
        self.node_bounds[q+'1'] = (xsplit,  b[1],     b[2],    ysplit)
        self.node_pts   [q+'2'] = set()
        self.node_bounds[q+'2'] = (b[0],    ysplit,   xsplit,  b[3])
        self.node_pts   [q+'3'] = set()
        self.node_bounds[q+'3'] = (xsplit,  ysplit,   b[2],    b[3])

        for pt in self.node_pts[q]:
            if   pt[0] <= xsplit and pt[1] <= ysplit:
                s = '0'
            elif                     pt[1] <= ysplit:
                s = '1'
            elif pt[0] <= xsplit:
                s = '2'
            else:
                s = '3'
            self.node_pts[q+s].add(pt)

        self.node_pts[q] = set()

    #
    # quad() - find the quad string describing where the point should reside
    #
    def quad(self, pt):
        s = ''
        while (s+'0') in self.node_pts.keys():
            b = self.node_bounds[(s+'0')]
            if   pt[0] <= b[2] and pt[1] <= b[3]:
                s += '0'
            elif                   pt[1] <= b[3]:
                s += '1'
            elif pt[0] <= b[2]:
                s += '2'
            else:
                s += '3'
        return s
    #
    # add() - add a list of points to the quadtree
    #    
    def add(self, pts):
        for pt in pts:
            q = self.quad(pt)
            self.node_pts[q].add(pt)
            if len(self.node_pts[q]) > self.max_pts_per_node:
                self.__split__(q)

    #
    # _repr_svg_()
    #
    def _repr_svg_(self):
        w     = h     = 600
        x_ins = y_ins = 5
        svg = []
        svg.append(f'<svg width="{w+2*x_ins}" height="{h+2*y_ins}">')
        xT = lambda x: x_ins + w * (x - self.bounds[0])/(self.bounds[2] - self.bounds[0])
        yT = lambda y: y_ins + h * (y - self.bounds[1])/(self.bounds[2] - self.bounds[1])
        for q in self.node_bounds.keys():
            if q+'0' not in self.node_bounds.keys():
                b = self.node_bounds[q]
                _color_ = rt.co_mgr.getColor(q)
                svg.append(f'<rect x="{xT(b[0])}" y="{yT(b[1])}" width="{xT(b[2])-xT(b[0])}" height="{yT(b[3])-yT(b[1])}" fill="{_color_}" opacity="0.4" />')
        for q in self.node_pts.keys():
            for pt in self.node_pts[q]:
                svg.append(f'<circle cx="{xT(pt[0])}" cy="{yT(pt[1])}" r="0.8" fill="#000000" />')
        svg.append('</svg>')
        return ''.join(svg)

qt = XYQuadTree((0.0, 0.0, 1.0, 1.0))
for s in range(10000):
    pt = (random.random(),random.random())
    qt.add([pt])
qt